<a href="https://colab.research.google.com/github/AlexanderPidodnya/MaisRecognition/blob/main/TrainAndCheckModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training model

In [ ]:
#only for colab
!pip uninstall tensorflow
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
#restart kernel

In [2]:
import tensorflow as tf
tf.__version__, tf.config.list_physical_devices('GPU')

('2.8.0', [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')])

In [1]:
import  os

In [2]:
custom_model_name = 'my_ssd_mobnet'
pretrained_model_name = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
pretrained_model_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
#pretrained_model_name = 'ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8'
#pretrained_model_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz'
tf_record_script_name = 'generate_tfrecord.py'
label_map_name = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH' : os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH' : os.path.join('Tensorflow', 'scripts'),
    'APIMODEL_PATH' : os.path.join('Tensorflow', 'models'),
    'ANNOTATION_PATH' : os.path.join('Tensorflow', 'workspace', 'annotations'),
    'IMAGE_PATH' : os.path.join('Tensorflow', 'workspace', 'images'),
    'PRETRAINED_MODEL_PATH' : os.path.join('Tensorflow', 'workspace', 'pre-trained-models'),
    'CHECKPOINT_PATH' : os.path.join('Tensorflow', 'workspace', 'models', custom_model_name),
    'OUTPUT_PATH' : os.path.join('Tensorflow', 'workspace', 'models', custom_model_name, 'export'),
    'TFJS_PATH' : os.path.join('Tensorflow', 'workspace', 'models', custom_model_name, 'tfjsexport'),
    'TFLITE_PATH' : os.path.join('Tensorflow', 'workspace', 'models', custom_model_name, 'tliteexport'),
    'PROTOC_PATH' : os.path.join('Tensorflow', 'protoc')
}

In [4]:
files = {
    'PIPELINE_CONFIG' : os.path.join('Tensorflow', 'workspace', 'models', custom_model_name, 'pipeline.config'),
    'TF_RECORD_SCRIPT' : os.path.join(paths['SCRIPTS_PATH'], tf_record_script_name),
    'LABELMAP' : os.path.join(paths['ANNOTATION_PATH'], label_map_name)
}

In [7]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'nt':
            !mkdir {path}
        if os.name == 'posix':
            !mkdir -p {path}


In [ ]:
paths

{'ANNOTATION_PATH': 'Tensorflow/workspace/annotations',
 'APIMODEL_PATH': 'Tensorflow/models',
 'CHECKPOINT_PATH': 'Tensorflow/workspace/models/my_ssd_mobnet',
 'IMAGE_PATH': 'Tensorflow/workspace/images',
 'OUTPUT_PATH': 'Tensorflow/workspace/models/my_ssd_mobnet/export',
 'PRETRAINED_MODEL_PATH': 'Tensorflow/workspace/pre-trained-models',
 'PROTOC_PATH': 'Tensorflow/protoc',
 'SCRIPTS_PATH': 'Tensorflow/scripts',
 'TFJS_PATH': 'Tensorflow/workspace/models/my_ssd_mobnet/tfjsexport',
 'TFLITE_PATH': 'Tensorflow/workspace/models/my_ssd_mobnet/tliteexport',
 'WORKSPACE_PATH': 'Tensorflow/workspace'}

 1. Download TF pretrained models from Tensorflow Model ZOO and instal TFOD

In [8]:
if os.name == 'nt':
    !pip install wget
    import wget

In [9]:
if not os.path.exists(os.path.join(paths['ANNOTATION_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 74457, done.
remote: Counting objects: 100% (477/477), done.
remote: Compressing objects: 100% (252/252), done.
remote: Total 74457 (delta 253), reused 427 (delta 215), pack-reused 73980
Receiving objects: 100% (74457/74457), 580.42 MiB | 32.84 MiB/s, done.
Resolving deltas: 100% (52742/52742), done.


In [10]:
#instal Tensorflow Object Detection
if os.name=='posix':
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.3 MB 8.6 MB/s 
     |███████████████████████████████

In [11]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

Running tests under Python 3.7.13: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-06-12 04:55:14.507908: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0612 04:55:14.781613 139723294300032 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.73s
I0612 04:55:15.223880 139723294300032 test_util.py:2459] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.73s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.58s
I0612 04:55:15.808911 139723294300032 test_u

In [12]:
import  object_detection

In [13]:
paths['PRETRAINED_MODEL_PATH'], pretrained_model_URL, pretrained_model_name

('Tensorflow/workspace/pre-trained-models',
 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz',
 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8')

In [14]:
if os.name =='posix':
    !wget {pretrained_model_URL}
    !mv {pretrained_model_name+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {pretrained_model_name+'.tar.gz'}
if os.name == 'nt':
    wget.download(pretrainde_model_URL)
    !move {pretrained_model_name+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {pretrained_model_name+'.tar.gz'}

--2022-06-12 04:55:58--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.152.128, 2607:f8b0:4001:c56::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.152.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.57M  --.-KB/s    in 0.1s    

2022-06-12 04:55:58 (192 MB/s) - ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’ saved [20518283/20518283]

ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu

# 2.Create Label Map

In [5]:
labels = [{'name' : 'mais', 'id':1},
          {'name' : 'dill', 'id':2},
          {'name' : 'buckwheat', 'id':3},
          {'name' : 'other', 'id':4}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item {\n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

#3.Create TF records

In [19]:
# OPTIONAL IF RUNNING ON COLAB
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
  !tar -zxvf {ARCHIVE_FILES}

In [16]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [17]:
resources = '/content/resources'
if not os.path.exists(resources):
  !mkdir {resources}
!git clone https://github.com/AlexanderPidodnya/MaisRecognition {resources}

Cloning into '/content/resources'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 96 (delta 3), reused 26 (delta 3), pack-reused 70
Unpacking objects: 100% (96/96), done.


In [ ]:
#coping from resources
import glob, shutil
onColab = os.name =='posix' 
if not onColab:
  res_img = r'C:\Users\AdminAsus\PycharmProjects\MaisRecognition\resources'
  train_path = os.path.join(paths['IMAGE_PATH'], 'train')
  test_path = os.path.join(paths['IMAGE_PATH'], 'test')
  if not os.path.exists(train_path):
      !mkdir {train_path}
  if not os.path.exists(test_path):
      !mkdir {test_path}
if onColab:
  res_img = r'/content/resources/resources'
  train_path = os.path.join(paths['IMAGE_PATH'], 'train')
  test_path = os.path.join(paths['IMAGE_PATH'], 'test')
  if not os.path.exists(train_path):
      !mkdir {train_path}
  if not os.path.exists(test_path):
      !mkdir {test_path}


imgs = []
rr = '.jpg'
for img in os.listdir(res_img):
  if rr in img.lower():
    imgs.append(img[:-4])
tr_k = len(imgs) * 0.8
for i, img in enumerate(imgs):
  fList = glob.glob(os.path.join(res_img, img+'*.*'))
  
  for f in fList:
    #print(f)
    if i<tr_k:
      if onColab:
        !cp {f} {train_path} 
      else:
        !copy {f} {train_path} 
    else:
      if onColab:
        !cp {f} {test_path}        
      else:
        !copy {f} {test_path}        
# may use a directive like !copy

In [20]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


#4.Copy Model Config to Training Folder

In [21]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], pretrained_model_name, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], pretrained_model_name, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

#5. Update Config for Transfer Learning

In [22]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [23]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config

{'eval_config': metrics_set: "coco_detection_metrics"
 use_moving_averages: false,
 'eval_input_config': label_map_path: "PATH_TO_BE_CONFIGURED"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED"
 },
 'eval_input_configs': [label_map_path: "PATH_TO_BE_CONFIGURED"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED"
 }
 ],
 'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.9969999790191

In [ ]:
files['PIPELINE_CONFIG']

'Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config'

In [24]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], pretrained_model_name, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                               f.write(config_text)

In [ ]:
pipeline_config

#6.Train the model

In [26]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [27]:
command = 'python {} --model_dir={} --pipeline_config_path={} --num_train_steps=20000'.format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'], files['PIPELINE_CONFIG'])
command

'python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=20000'

In [ ]:
#optional

!pip install gin-config==0.1.1

In [ ]:
#optional
!pip install TensorFlow-addons

In [29]:
!pip install opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
pip uninstall opencv-python-headless

Found existing installation: opencv-python-headless 4.6.0.66
Uninstalling opencv-python-headless-4.6.0.66:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.6.0.66.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavcodec-5896f664.so.58.134.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavformat-8ef5c7db.so.58.76.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavutil-9c768859.so.56.70.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libbz2-a273e504.so.1.0.6
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libcrypto-d21001fc.so.1.1
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libgfortran-91cc3cb1.so.3.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libopenblas-r0-f650aae0.3.3.so
    /usr/local/lib/python3.7/dis

In [31]:
pip install opencv-python-headless==4.5.2.52

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 38.2 MB 1.2 MB/s 


In [32]:
!{command}

2022-06-12 05:13:25.642819: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0612 05:13:25.651125 140563238905728 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 20000
I0612 05:13:25.658090 140563238905728 config_util.py:552] Maybe overwriting train_steps: 20000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0612 05:13:25.658411 140563238905728 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0612 05:13:25.961495 140563238905728 deprecation.py:356] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experiment

#8. Load Train Model from CheckPoint

In [ ]:
!pip install tensorflow-object-detection-api

In [6]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

AttributeError: ignored

In [34]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-21')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

NameError: ignored